In [ ]:
# Code based on souce from https://machinelearningmastery.com/how-to-develop-a-pix2pix-gan-for-image-to-image-translation/

# Required imports for dataset import, preprocessing and compression

"""
GAN analysis file. Takes in trained .h5 files created while training the network.
Generates test files from testing synthetic input photos (files the GAN has never seen before).
Generates psnr and ssim ratings for each model/.h5 files and loads the results into excel files.
"""

from os import listdir
import numpy
from numpy import asarray
from numpy import vstack
from numpy import savez_compressed
from numpy import load
from numpy import expand_dims
from numpy.random import randint
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import load_model
import matplotlib
from matplotlib import pyplot
import glob


In [ ]:
# Load images from a directory to memory
def load_images(path, size=(256,256)):
    pic_list = list()

    # enumerate filenames in directory, assume all are images

    for filename in listdir(path):

        # load and resize the image (the resizing is not being used in our implementation)
        pixels = load_img(path + filename, target_size=size)

        # convert to numpy array
        pic_list.append(img_to_array(pixels))

    return asarray(pic_list)

# Load and prepare test or validation images from compressed image files to memory

def load_numpy_images(filename):

    # Load the compressed numpy array(s)
    data = load(filename)

    img_sets =[]

    for item in data:
        img_sets.append((data[item]- 127.5) / 127.5)

    return img_sets

# Plot source, generated and target images all in one output

def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0

    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    for i in range(len(images)):
        # define subplot
        pyplot.subplot(1, 3, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(images[i])
        # show title
        pyplot.title(titles[i])
    pyplot.show()

# Load a single image
def load_image(filename, size=(256,256)):
    # load image with the preferred size
    pixels = load_img(filename, target_size=size)
    
    # convert to numpy array
    pixels = img_to_array(pixels)
    
    # scale from [0,255] to [-1,1]
    pixels = (pixels - 127.5) / 127.5
    
    # reshape to 1 sample
    pixels = expand_dims(pixels, 0)
    
    return pixels


In [ ]:
####################
# Convert the training dataset to a compressed numpy array (NOT USED FOR METRICS)
####################

# Source images path (synthetic images)
path = 'data/training/synthetic/'
src_images = load_images(path)

# Ground truth images path
path = 'data/training/gt/'
tar_images = load_images(path)

# Perform a quick check on shape and sizes
print('Loaded: ', src_images.shape, tar_images.shape)

# Save as a compressed numpy array
filename = 'data/training/train_256.npz'
savez_compressed(filename, src_images, tar_images)
print('Saved dataset: ', filename)



In [ ]:
###################
# Convert the validation dataset to a compressed numpy array (.npz)
###################

# Source images path
path = 'data/validation/synthetic/'
src_images = load_images(path)

# Ground truth images path
path = 'data/validation/gt/'
tar_images = load_images(path)

# Perform a quick check on shape and sizes
print('Loaded: ', src_images.shape, tar_images.shape)

# Save as a compressed numpy array
filename = 'data/validation/validation_256.npz'
savez_compressed(filename, src_images, tar_images)
print('Saved dataset: ', filename)


In [ ]:
# Load the validation dataset from the compressed numpy array to memory

img_sets = load_numpy_images('data/validation/validation_256.npz')
src_images = img_sets[0]
print('Loaded: ', src_images.shape)

#tar_images = img_sets[1]
#print('Loaded: ', tar_images.shape)

# Gain some memory
del img_sets



In [ ]:
# Get the list of gt image names so outputs can be named correctly
path = 'data/validation/gt/'
img_list = os.listdir(path)

exp_path = 'models/exp6/'
model_list = os.listdir(exp_path)

# loop through model/.h5 files 
for model in model_list:
    model_dir = 'outputs/'+model[:-3]
    os.mkdir(model_dir)

    # load model weights to be used in the generator
    predictor = load_model(exp_path+model)
    names = 0

    for i in range(0, len(src_images),10 ):
        # push image through generator
        gen_images = predictor.predict(src_images[i:i+10])
        
        # name and export file
        for img in range(len(gen_images)):
            filename = model_dir+'/'+img_list[names]
            names += 1
            matplotlib.image.imsave(filename, (gen_images[img]+1)/2.0)




In [ ]:
# Code to evaluate generated images from each model run for PNSR and SSIM

import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import re
import cv2
import pandas as pd

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

exp_dir = 'outputs/' # result director
gt_dir = 'data/validation/gt/' # ground truth directory
img_list = os.listdir(gt_dir)

column_names =[]
exp_list = [ f.name for f in os.scandir(exp_dir) if f.is_dir() ]
for exp in exp_list:

    model_list = [ f.name for f in os.scandir('outputs/'+exp+'/') if f.is_dir() ]
    
    for model in model_list:
            column_names.append(exp+'_'+model)
# create data frames for excel output
psnr_df = pd.DataFrame(columns = column_names)
ssim_df = pd.DataFrame(columns = column_names)

i=0
psnr_master=[]
ssim_master=[]
for img in img_list:  # loop through every image created by the generator
    i+=1
    # load image and create a grayscale for ssim measurement
    gt = cv2.imread(gt_dir+img)
    gt_gray = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)

    psnr_list=[]
    ssim_list =[]

    exp_list = [f.name for f in os.scandir(exp_dir) if f.is_dir()]
    # for each experiment
    for exp in exp_list:
        
        model_list = [ f.name for f in os.scandir('outputs/'+exp+'/') if f.is_dir() ]
        # for each generator weights/model (outputted h5 file from experiemnt)
        for model in model_list:
            pred = cv2.imread(exp_dir+exp+'/'+model+'/'+img)
            pred_gray = cv2.cvtColor(pred, cv2.COLOR_BGR2GRAY)
            
            # calculate psnr and ssim
            psnr_list.append(psnr(gt, pred, data_range=pred.max() - pred.min()))
            ssim_list.append(ssim(gt_gray, pred_gray, data_range=pred.max() - pred.min()))
            

    psnr_master.append(psnr_list)
    ssim_master.append(ssim_list)

# export for excel use   
psnr_df = pd.DataFrame(psnr_master, columns = column_names)
psnr_df.index = img_list
psnr_df.to_csv("PSNR.csv")

ssim_df = pd.DataFrame(ssim_master, columns = column_names)
ssim_df.index = img_list
ssim_df.to_csv("SSIM.csv")

In [ ]:
import PIL


['sidewalk winter -grayscale -gray_05189.jpg', 'sidewalk winter -grayscale -gray_07146.jpg', 'snow_animal_00447.jpg', 'snow_animal_03742.jpg', 'snow_intersection_00058.jpg', 'snow_nature_1_105698.jpg','snow_nature_1_108122.jpg','snow_nature_1_108523.jpg','snow_walk_00080.jpg','winter intersection -snow_00399.jpg','winter__street_03783.jpg','winter__street_05208.jpg']
pic_list_dims = [(426, 640), (538, 640), (640, 427), (432, 640), (480, 640), (640, 527), (480, 640), (427, 640), (640, 427), (502, 640), (269, 640), (427, 640)]

i=0

# load an image
def load_image(filename, size=(256,256)):
	# load image with the preferred size
	pixels = load_img(filename, target_size=size)
	# convert to numpy array
	pixels = img_to_array(pixels)
	# scale from [0,255] to [-1,1]
	pixels = (pixels - 127.5) / 127.5
	# reshape to 1 sample
	pixels = expand_dims(pixels, 0)
	return pixels

src_path = 'data/realistic_full/'
src_filename = pic_list[i]
src_image = load_image(src_path+src_filename)

print('Loaded', src_image.shape)

model_path = 'models/Experiment4/'
model_filename = 'model_125000.h5'

predictor = load_model(model_path+model_filename)
gen_img = predictor.predict(src_image)

# scale from [-1,1] to [0,1]
gen_img = (gen_img[0] + 1) / 2.0
# plot the image
pyplot.imshow(gen_img)
pyplot.axis('off')
pyplot.show()

gen_path = 'final/'
gen_filename = src_filename

matplotlib.image.imsave(gen_path+gen_filename, gen_img)
gen_img = load_img(gen_path+gen_filename, target_size=pic_list_dims[i])
pyplot.imshow(gen_img)
pyplot.axis('off')
pyplot.show()
print(gen_img)
gen_img.save(gen_path+gen_filename)

In [ ]:
pic_list = ['sidewalk winter -grayscale -gray_05189.jpg', 'sidewalk winter -grayscale -gray_07146.jpg', 'snow_animal_00447.jpg', 'snow_animal_03742.jpg', 'snow_intersection_00058.jpg', 'snow_nature_1_105698.jpg','snow_nature_1_108122.jpg','snow_nature_1_108523.jpg','snow_walk_00080.jpg','winter intersection -snow_00399.jpg','winter__street_03783.jpg','winter__street_05208.jpg']
src_path = 'data/realistic_full/'

dims=[]
for img in pic_list:
    pixels = load_img(src_path+img)
    dims.append(tuple(reversed(pixels.size)))
print(dims)   